In [10]:
%matplotlib inline
import numpy as np
import networkx as nx
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm
from bokeh.io import output_file, show
from bokeh.plotting import figure, from_networkx
import datashader as ds
import datashader.transfer_functions as tf

from bokeh.models import Ellipse, Circle, StaticLayoutProvider, MultiLine, HoverTool, BoxZoomTool, ResetTool, EdgesAndLinkedNodes, BoxSelectTool, TapTool, NodesAndLinkedEdges
from bokeh.palettes import Spectral4

In [2]:
graph = nx.read_gpickle("../../data/processed/umda_graph.gpkl.gz")

In [4]:
# locs = {key: (node["x"], node["y"]) for key, node in graph.nodes.items()}
data = [[index, node["x"], node["y"], node["source"]] for index, node in graph.nodes.items()]

In [123]:
df = pd.DataFrame(data, columns=["id", "x", "y", "source"])
df.set_index("id", inplace=True)

In [124]:
df.loc[df["source"] != 3, "source"] = 0.

In [125]:
df["source"] = df["source"].astype("category")

In [7]:
def bokehify_graph(nx_graph, title: str, path: str, show_plot: bool = True, locs=None):
    if not locs:
        locs = nx.spring_layout
    plot = figure(title=title, x_range=(-1.1,1.1), y_range=(-1.1,1.1), plot_width=1000, plot_height=700)

    node_hover_tool = HoverTool(tooltips=[("A", "@A"), ("B", "@B"), ("Log10 rate", "@rate")])
    plot.add_tools(node_hover_tool, TapTool(), BoxZoomTool(), ResetTool(), BoxSelectTool())

    graph_renderer = from_networkx(nx_graph, locs, scale=2, center=(0,0), seed=42)

    graph_renderer.node_renderer.glyph = Circle(size=5, fill_color="source")
    graph_renderer.node_renderer.selection_glyph = Circle(size=15., fill_color=Spectral4[2])
#     graph_renderer.edge_renderer.glyph = MultiLine(line_color="react_color", line_alpha=0.3, line_width=0.5)
#     graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color='#abdda4', line_width=3, line_alpha=0.6)
#     graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=3, line_alpha=0.8)

    graph_renderer.inspection_policy = EdgesAndLinkedNodes()
    graph_renderer.selection_policy = NodesAndLinkedEdges()
    plot.renderers.append(graph_renderer)

    output_file(f"{path}.html")
    if show_plot:
        show(plot)
    return plot

In [7]:
# bokehify_graph(graph, "UMAP inventory", "umap_full", locs=locs)

In [47]:
from palettable.cartocolors.qualitative import Pastel_8

In [138]:
def nodesplot(nodes, name=None, canvas=None, cat=None, **kwargs):
    canvas = ds.Canvas(**kwargs) if canvas is None else canvas
    aggregator = None if cat is None else ds.count_cat(cat)
    agg = canvas.points(nodes,'x','y', aggregator)
    tmc1_mols = canvas.points(nodes.loc[nodes["source"] == 3.], "x", "y", aggregator)
    all_data = tf.spread(tf.shade(agg, cmap=Pastel_8.hex_colors, color_key=Pastel_8.hex_colors, alpha=1, min_alpha=5), px=1, name=name)
    tmc1_data = tf.spread(tf.shade(tmc1_mols, alpha=255, min_alpha=255), px=3)
    return tf.stack(all_data, tmc1_data, how="over", name=name)

In [142]:
nodesplot(df, **{"plot_height": 600, "plot_width": 600})

TypeError: ufunc 'over' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
import networkx as nx

def ng(graph,name):
    graph.name = name
    return graph

def nx_layout(graph):
    layout = nx.circular_layout(graph)
    data = [[node]+layout[node].tolist() for node in graph.nodes]

    nodes = pd.DataFrame(data, columns=['id', 'x', 'y'])
    nodes.set_index('id', inplace=True)

    edges = pd.DataFrame(list(graph.edges), columns=['source', 'target'])
    return nodes, edges

def nx_plot(graph, name=""):
    print(graph.name, len(graph.edges))
    nodes, edges = nx_layout(graph)
    
    direct = connect_edges(nodes, edges)
    bundled_bw005 = hammer_bundle(nodes, edges)
    bundled_bw030 = hammer_bundle(nodes, edges, initial_bandwidth=0.30)

    return [graphplot(nodes, direct,         graph.name),
            graphplot(nodes, bundled_bw005, "Bundled bw=0.05"),
            graphplot(nodes, bundled_bw030, "Bundled bw=0.30")]